---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:

def get_list_of_university_towns():
    ins='''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df=pd.DataFrame([],columns=['State','RegionName'])
    with open('university_towns.txt','r') as f:
        st = ''
        city = ''
        for line in f:
            if '[edit]' in line:
                st = line[:line.find('[')].strip()
                continue
            city = line 
            if '(' in city.strip():
                city = line[:city.find('(')]
                city = city.strip()
            df = df.append(pd.DataFrame([[st,city]],columns=['State','RegionName']),ignore_index=True)
    
                
            
       
                
                
                
        
                
                
    
    
                
                
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def read_gdp_file():
    df = pd.read_excel('gdplev.xls')
    df = df.drop(['Current-Dollar and "Real" Gross Domestic Product','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5'],axis=1)
    df = (df.drop([0,1,2,3,4,5,6])
          .rename(columns={'Unnamed: 4': 'Quarter','Unnamed: 6': 'GDP'})
          .set_index('Quarter')
         )
    find2000 = df.index.get_loc('2000q1')
    df = df.ix[find2000:]
    
    return df
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = read_gdp_file()
    offset = len(df)-1
    for x in range(1, offset):
        if (df.iloc[x]['GDP'] < df.iloc[x - 1]['GDP']) and (df.iloc[x + 1]['GDP'] < df.iloc[x]['GDP']):
            return df.iloc[x].name
    
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    df = read_gdp_file()
    start = get_recession_start()
    for x in range (df.index.get_loc(start),len(df)):
        if (df.iloc[x]['GDP']>df.iloc[x-1]['GDP']) and (df.iloc[x-1]['GDP'] > df.iloc[x-2]['GDP']):
            return df.iloc[x].name

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = read_gdp_file()
    start = df.index.get_loc(get_recession_start())
    end = df.index.get_loc(get_recession_end())
    df2 = df['GDP'][start:end]
    low = df[df['GDP']==np.min(df2)].iloc[0]
    
    
    return low.name
get_recession_bottom()


'2009q2'

In [7]:
def quarter(y, m):
    if m <= 3:
        q = 1
    elif m <= 6:
        q = 2
    elif m <= 9:
        q = 3
    elif m <= 12:
        q = 4
    return (str(y) + 'q' + str(q))

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df = (df.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
          .replace({'State':states})
          .set_index(['State','RegionName'])
         )
    df = df.convert_objects(convert_numeric=True)
    df = df.replace(to_replace='NaN',value=np.NaN)
    
    df = df.sort()
    cutter = list(df.columns.values)
    cutter = cutter.index('2000-01')
    df = df.drop(df.columns[:cutter],axis=1)
    leng = len(df.columns)
    x = 0
    while x < leng:
        col = df.iloc[:, x].name
        y = col.split('-')[0]
        m = col.split('-')[1]
        y = int(y)
        m = int(m)
        q = quarter(y, m)
        if x + 3 < leng:
            split = df.iloc[:, x:x + 3]
        else:
            split = df.iloc[:, x:x]
        df[q] = split.mean(axis=1)
        x += 3
    df = df.drop(df.columns[:x], axis=1)
    return df 
convert_housing_data_to_quarters()

2000q2         2000q3         2000q4  \
State     RegionName                                                     
Alabama   Adamsville        69166.666667   69800.000000   71966.666667   
          Alabaster        123066.666667  123166.666667  123700.000000   
          Albertville       72600.000000   72833.333333   74200.000000   
          Arab              81566.666667   81333.333333   82966.666667   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       78533.333333   79133.333333   81300.000000   
          Bayou La Batre    44500.000000   44266.666667   43666.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54400.000000   54966.666667   56066.666667   
          Boaz              70266.666667   70300.000000   71466.666667   
          Brent             94333.333333   96166.666667   98333.333333   
          Brighton                   NaN            NaN            NaN   
          Brookwood         95100.000000   98866.666667   99966.666667   
          Buhl              94600.000000   96500.000000   96566.666667   
          Calera           110366.666667  108000.000000  107933.333333   
          Center Point      81233.333333   81500.000000   82233.333333   
          Centreville       96566.666667   98000.000000   99366.666667   
          Chalkville        94433.333333   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          167033.333333  166900.000000  167400.000000   
          Chickasaw         53666.666667   54933.333333   56066.666667   
          Chunchula         81766.666667   82200.000000   83400.000000   
          Citronelle        66633.333333   68066.666667   67766.666667   
          Clay             122266.666667  123966.666667  126500.000000   
          Coden             64800.000000   66866.666667   68566.666667   
          Coker            120766.666667  118166.666667  115333.333333   
          Concord           78700.000000   80133.333333   82800.000000   
          Cottondale       102633.333333  104766.666667  105300.000000   
...                                  ...            ...            ...   
Wisconsin Vernon           178200.000000  174300.000000  177466.666667   
          Vienna           181533.333333  182433.333333  186300.000000   
          Vinland          126766.666667  134933.333333  137833.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        119433.333333  120200.000000  121966.666667   
          Waukesha         138433.333333  136733.333333  139833.333333   
          Waunakee         191433.333333  192666.666667  193733.333333   
          Waupun            84000.000000   84400.000000   84533.333333   
          Wausau            69466.666667   71033.333333   69833.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        81933.333333   84066.666667   80666.666667   
          Weston            82700.000000   84166.666667   85800.000000   
          Wheatland        132133.333333  134933.333333  137833.333333   
          Whitelaw         101433.333333  108033.333333  114000.000000   
          Williams Bay     115300.000000  110766.666667  110700.000000   
          Wilson           129366.666667  132433.333333  137200.000000   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        123666.666667  124133.333333  126066.666667   
          Wind Point       145266.666667  140866.666667  147866.666667   
          Wisconsin Dells  103400.000000  105000.000000  105433.333333   
          Wolf River        99333.333333  103800.000000  107633.333333   
          Woodruff                   N

In [17]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df = convert_housing_data_to_quarters()
    towns = get_list_of_university_towns()
    towns = towns.set_index(['State','RegionName'])
    start = df.columns.get_loc(get_recession_start())
    low = df.columns.get_loc(get_recession_bottom())
    df['rat'] = df.iloc[:,start - 1]/df.iloc[:,low]
    df = pd.DataFrame(df.loc[:,'rat'])
    uni = pd.merge(df,towns,how='inner',left_index=True,right_index=True)
    uni = uni.dropna()
    non = pd.merge(df,towns,how='outer',left_index=True,right_index=True,indicator=True)
    non = non[non['_merge']=='left_only']
    non = non.dropna()
    s, p = ttest_ind(uni['rat'], non['rat'])
    return True, p, 'university town'
run_ttest()

(True, 0.002724063704761164, 'university town')